# Word Embedding

> the collective name for a set of language modeling and feature learning techniques in NLP where words and phrases from the vocabulary are mapped to vectors of real numbers. (from Wikipedia)

> 수학적으로, 고차원의 공간을 더 낮은 공간으로 변환하는 방법(embedding)과 같은 의미이기도 하다.

> 결국, 고차원으로 표현된 feature vector(local representation, BOW, TF-IDF 등)을 distributional semantic을 가지는 vector space에 mapping 시켜주는 방법이다.

> <b>"You shall know a word by the company it keeps"(John R. Firth, 1957)<b>, it called "Distributed Hypothesis"

![word embedding fig](7.PNG)

![visualize word vectors](8.PNG)

다음은 최근 많이 쓰이는 word embedding 방법들이다.

wevi : word embedding visual inspector
    
> https://ronxin.github.io/wevi/

이론을 정리하기 위해 체험을 해보자.

## 1. Word2Vec

> 현재 word embedding이 핫하게 된 시작 알고리즘. "Distributed representations of words and phrases and their compositionality(NIPS 2013)" 에 처음 소개되었다.

> Reference : https://code.google.com/archive/p/word2vec/

![skip-gram](9.PNG)
![simple-skip-gram](10.png)

> Skip Gram : 어떤 target 단어(학습을 하고 싶은 단어)와 주변 단어(context word)를 설정해서 함께 등장하는 단어들이 있을 텐데, 정말 함께 등장 했는지 안 했는지를 binary classification으로 해결하는 것
- 이 단어와 함께 나오는 단어들이 얼마나 높은 확률로 등장하는 지 학습하는 것
- 같이 등장하는 것을 positive sample, 같이 등장하지 않는 것을 negative sample이라고 정의
   
> Skip Gram할 때 윈도우라는 개념있음.
- 윈도우: 문장에 단어들이 쭉 있으면 이 단어들을 내가 한번에 눈으로 보는 단위 정도
- 문장을 쭉 볼 때 맨 앞부터 맨 뒤까지 신경쓰면서 읽진 않지. 문맥이라는 걸 파악하는 어느정도의 단위를 윈도우라고 정의함. 함께 나온다/안 나온다를 정의 하는 대상의 규모가 윈도우가 됨
- 윈도우 안에 있으면 함께 나옴 / 윈도우가 밖에 있으면 함께 안 나옴
- 윈도우 크기는 사용자가 정의해줘야하고, 크기에 따라 문맥이라는 걸 파악하는 길이가 짧아질 수도 있고 길어질 수도 있음

> Skip-Gram with Negative Sampling, 줄여서 SGNS라고 부르며 Neural Net을 이용한 word embedding이 빠르게 구현가능해진 이유기도 하다.

> Negative Sampling이란, 마지막 단계의 softmax를 구하는 문제를 주변 단어(postive class)와 무작위로 골라진 나머지 단어들(negative class)로 분류하는 binary classfication 문제로 바꿔주는 기법이며, 이를 통해 굉장히 빠르게 word embedding 수행이 가능하다.
- Negative Sampling 방법은 계산량을 많이 줄여줌. 강의자료에 나왔던 nplm이라는 개념은 softmax를 해서 이 단어가 빈칸에 등장할 확률을 max가 되게끔 학습을 해주는데 이게 단어 전체를 softmax 계산해야해서 굉장히 오래걸려
- 이 친구는 softmax 계산을 매번 다 안하고 Negative Sampling이라는 방법을 사용해서 이 단어들이 동시 등장했다면 두 단어들은 가까우니 두 단어의 dot product(내적)값이 커지게, 서로가 변환공간에서 가까워지게끔 학습
- 두 단어가 Negative Sampling이면 멀어지게끔 loss function 정의해서 학습하고 loss function이 최소가 되게끔

In [1]:
import nltk
nltk.download('movie_reviews') # 간단한 영문 데이터 셋
nltk.download('punkt')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\영현\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\movie_reviews.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\영현\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [2]:
from nltk.corpus import movie_reviews
sentences = [list(sent) for sent in movie_reviews.sents()]
sentences[0] # input이 됨

['plot',
 ':',
 'two',
 'teen',
 'couples',
 'go',
 'to',
 'a',
 'church',
 'party',
 ',',
 'drink',
 'and',
 'then',
 'drive',
 '.']

In [3]:
# gensim에서 word2vec 불러오기
from gensim.models import Word2Vec

# gensim (generator similar) : 토픽 모델링를 위해 처음 등장했지만, 지금은 word2vec이나 fasttext같은 걸로 훨씬 많이 쓰임
# 지금은 자연어처리의 보명적인 library가 되기위해 여러가지 구현되었음
# 특히 word embedding 파트에서 강력하게 이것저것 구현되어있음

# 학습 모델 구현.

# Word2Vec의 data input 형태는 list of list of word
# 토큰 단위로 되어있는 리스트 1개가 문장이고, 이 문장들이 들어있는 list가 input이 됨
w2v_model = Word2Vec(sentences, min_count=5, size=300, sg=1, iter=10,
                     workers=4, ns_exponent=0.75, window=7)
# min_count=5: 최소단어가 5개 이하면 쓰지 않겠다
# size : hidden layer의 노드 개수
# sg=1 : Skip Gram 쓰겠다
# iter : iteration이 많으면 많을 수록 좋겠지만 지금은 간단하게
# workers : 실제 CPU를 몇개 쓸거냐
# window=7 : 내 기준으로 앞,뒤 7개를 보겠다 => 나 기준 왼쪽으로 3개 오른쪽으로 3개를 동시에 보겠다

In [5]:
# 모델 평가, 비슷한 단어 찾기.
w2v_model.wv.most_similar('good') 
# word2vec가 학습이 다 되면 embedding 공간이 생기고, 그 embeddin 공간에 있는 어떤 단어와 가장 비슷한 단어 찾아줌
# 비슷하다는 기준은 cosine similarity가 됨
# word2vec는 애초에 학습할 때 어떤 단어가 가깝다라는 걸 cosine similarity를 가정하기 때문

[('decent', 0.525941014289856),
 ('lousy', 0.4812678098678589),
 ('pitiful', 0.46957242488861084),
 ('passable', 0.46780040860176086),
 ('glorified', 0.4646782875061035),
 ('darned', 0.4630981981754303),
 ('gutsy', 0.46071916818618774),
 ('milestone', 0.458088755607605),
 ('commendable', 0.45794373750686646),
 ('dopey', 0.45742207765579224)]

<학습결과> - good이랑 관련없는 단어들이 나왔음. 왜 그런가 이유가 가장 중요
</br>
- word2vec는 비슷하게 같이 등장하면 얘네들의 학습을 높게 해줌
- 감정표현이 나오는 문장에서 예를 들어, i am good 이라고 하면 i am ___ 이라는 건데, 그 빈칸 앞 뒤가 비슷했다는거지 (i am bad, i am decent) (아까 학습원리를 되돌아보면)
- good과 비슷한 곳에 형용사가 등장했다는 것을 알 수 있고, 기분을 나타내는 형용사가 나왔는데 good과 비슷한 단어는 나오질 않았지
- 하이퍼파라미터 조정을 좀 더 해봐야하고, good처럼 긍정적인 단어가 많이 들어있는 corpus를 더 넣어준다는 등의 전처리 필요

## 2. GloVe

> GloVe는 Global Vectors의 약자로, aggregated global word global co-occurrence statistics를 최적화하는 방향으로 학습하는 word embedding 방법이다. "GloVe: Gloval Vectors for Word Representation(EMNLP 2014)"에 소개되었다.

> Reference : https://nlp.stanford.edu/projects/glove/

- word2vec는 윈도우 크기 내에 있는 단어들만 확률적으로 높게 계산되게끔 학습이 된다고 했는데, 그 말은 local 정보를 사용하겠다는 것(Global 정보 빠져있음)
- 그래서 GloVe는 Global 정보를 사용하겠다고해서 word에 Global co-occurence의 등장 확률 값이 높게끔 학습
- 밑의 식이 loss function
- 밑의 metrix 값과 동시등장하는 두 단어의 dot product값이 서로 비슷해지게, 내가 지금 보고있는 target 단어와 주변 단어가 global co-occurrence값이 높으면 이 두 단어의 dot product가 행렬 분해한 값과 비슷해지게 끔 학습 <= 밑의 식의 해석

![glove](11.png)

In [ ]:
# GloVe는 현재 파이썬으로는 구현이 힘듭니다(업데이트가 안돼서)
# 사용을 위해서는 c++을 사용하셔야 합니다.

#from glove import Glove

#glove_model = Glove(no_components=100, learning_rate=0.05)
#glove_model.fit(sentences, epochs=10, no_threads=4, verbose=True)

## 3. FastText

> 현재 NLP task에서 word embedding의 baseline으로 사용되는 기법이다. subword embedding model, char n-gram embedding model이라고도 한다.

> word2vec을 만들었던, Tomas Mikolov가 Google에서 Facebook으로 옮긴 뒤에 낸 모델로 word2vec의 단점을 보완한 모델이다.

> word2vec의 단점이었던, OOV(Out Of Vocabulary) 문제와 low frequency를 많이 해결하였다.
- OOV : 학습 corpus에 없었던 단어는 word2vec로 표현이 안되는 문제(반영하려면 다시 전체 학습을 시켜줘야해서 복잡했음)
- low frequency: word2vec는 근본적으로 positive/negative sampling 방법을 사용할 때 단어의 등장 횟수 자체가 적으면 무시되는 경향이 컸음

> word를 subword 단위로 표현하는 것으로 기본적으로 SGNS 방식이다.

> Reference : https://fasttext.cc/

![char3-grams](12.png)
![char3-grams](13.png)

> characet N-grams 이라는 방법 사용하게됨
- going이라는 단어를 N=3을 적용, <go와 ng>에 나오는 꺽쇠(<,>)는 단어의 시작과 끝을 알림 (go와 ng라는 단어가 중간에 등장할 수도 있기때문에 차이를 주기위해서)
- 그래서 going에 <,>을 추가해서 7개를 연달아 3개씩 나눠
- 나눠진 각 sub component는 +라는 concatnate 연산돼서 going이라는 벡터에 가까워지게끔 학습됨
- 물론 각 sub component 토큰들은 다른 단어에서도 쓰이겠지. 그러면 거기에도 가까워지게끔 매번 학습됨
- 그래서 각 sub component들이 본래의 의미를 가질 수 있게끔

> 실제 한글의 특징 때문에 한글에서 많이 쓰임. 한글의 자소 분리(초/중/종성)를 n-grams으로 짤라서 사용

In [7]:
# gensim에서 FastText 불러오기
from gensim.models import FastText

# FastText 학습.
fast_model = FastText(sentences, min_count=5, sg=1, size=300, workers=4,
                      min_n=2, max_n=7, alpha=0.05, window=7, iter=10)
# min_n/max_n : character n-gram을 실제로 할 땐 n을 고정시키는 게 아니고 최대/최소 개수를 주고 그것들의 ramdon sampling을 하게끔 만듦
# 이러면 다양한 학습이 될 수 있고, 똑같은 단어에서 정보를 많이 뽑아낼 수 있음

fast_model.save('fast_model')

In [9]:
# 결과 평가.
fast_model.wv.most_similar('good') 

[('goods', 0.5055395364761353),
 ('goodnight', 0.43297654390335083),
 ('great', 0.41480714082717896),
 ('goofball', 0.40592318773269653),
 ('goose', 0.39415624737739563),
 ('bad', 0.3819223642349243),
 ('marvellous', 0.36979711055755615),
 ('goo', 0.35548409819602966),
 ('gutsy', 0.34930139780044556),
 ('glorified', 0.3475170135498047)]

<학습결과> - 아까랑 다르게 good이랑 관련된 단어들이 많이 나왔음
</br>
- good이라는 sub word들을 학습했기때문에 good의 sub component들이 있는 단어들의 학습이 잘 되겠지

## 4. ELMo

> ELMo는 Embeddings from Language Model의 약자입니다. ELMo는 **pre-trained language model**을 사용하여 문맥에 맞는 word embedding, "Contextualized Word Embedding"을 만드는 방법입니다.
- 사용자가 어떤 주어진 데이터에 대해서 학습하는 데이터에 따라 다른 맥락들을 가지고 학습
- fasttext, word2vec, glove는 사실 통계 정보(동시등장확률)만 썼었는데, ELMo는 동시등장을 좀 더 다르게 학습하겠다, Contextual하게 학습하겠다라는 것

> bidirectional Language Model을 이용하여, pre-trained embedding vector를 corpus의 context(syntax, semantics, polysemy) 정보를 보완해주는 embedding vector를 만들어 준다.

> tensorflow, pytorch를 통해서 bidirectional LSTM model을 만들어 사용이 가능하다. (이미 구현된 model이 github에 공개되어있다.)

> "Deep contextualized word representations(NAACL 2018)"에 소개된 방법입니다.

In [9]:
#!pip install tensorflow_hub

In [10]:
# tensorflow_hub에서 데이터 불러오기.
import tensorflow_hub as hub

# 가까운 미래에는 실행이 되길 빌면서..
elmo = hub.Module('https://tfhub.dev/google/elmo/3', trainable=True)

embeddings = elmo(
    ["the cat is on the mat", "dogs are in the fog"],
    signature="default",
    as_dict=True)["elmo"]

RuntimeError: Exporting/importing meta graphs is not supported when eager execution is enabled. No graph exists when eager execution is enabled.

- tf 2.0 버전에는 ELMo가 안 들어가있음
- tf 1.x 에서 실행하면 ELMo 쓸 수 있음

In [11]:
print(embeddings)

Tensor("module_apply_default/aggregation/mul_3:0", shape=(2, 6, 1024), dtype=float32)


> Reference : https://allennlp.org/elmo, https://github.com/allenai/bilm-tf

- ELMo는 LSTM이 2개가 들어있음. 순방향 1개, 역방향 1개로 붙어있음
- 처음에 ELMo로 embedding 벡터들이 쭉 들어오는데 여기있는 embedding 벡터들은 앞에서 봤던 word2vec나 fasttext가 아니고 그냥 글자 하나하나(유니코드값으로 변환된 게 input으로 들어옴)
- forward network 쪽에는 순서대로 들어오고, backward network 쪽에는 반대로 들어옴. 말 그대로 문장이라는 것을 앞에서 한번, 뒤에서 한번 양쪽방향으로 문맥을 학습하기위해. 그래서 LSTM 2개가 따로따로 학습된 다음에 그 결과들을 ELMo 임베딩이라는 걸로 맨 위에서 합쳐주는 것으로 실제 결과 임베딩 값이 나오게 됨
- LSTM도 여러 개 layer라서 각 layer에 따라 각 단어들이 얼마나 가중치를 가지는지 학습됨

![elmo_architecture](14.png)
![elmo_architecture](15.png)